# 基于机器学习数据库飞速上线AI应用——RUL

剩余使用寿命（remaining useful life，RUL），指一个系统正常工作一段时间后,能够正常运转的时间。借助RUL,工程师可以安排维护时间、优化运行效率并避免计划外停机。因此,预测RUL是预测性维护计划中的首要任务。 
本次的任务就是开发一个通过机器学习模型进行剩余使用寿命预测的实时智能应用。我们使用NASA提供的[Turbofan Engine Degradation Simulation Data Set](https://ti.arc.nasa.gov/tech/dash/groups/pcoe/prognostic-data-repository/#turbofan)，作为训练集与测试集。

整个应用开发是基于[notebook](http://ipython.org/notebook.html)。


## 初始化环境
整个初始化过程包含安装fedb，以及相关运行环境，初始化脚步可以参考https://github.com/4paradigm/DemoApps/blob/main/predict-remaining-useful-life-nb/demo/init.sh

In [ ]:
!cd demo && sh init.sh

## 导入历史数据到fedb

使用fedb进行时序特征计算是需要历史数据的，所以我们将历史数据导入到fedb，以便实时推理可以使用历史数据进行特征推理，导入代码可以参考https://github.com/4paradigm/DemoApps/blob/main/predict-taxi-trip-duration-nb/demo/import.py 。

这里使用`test_FD004.txt`作为历史数据，有多个engine多次cycle的检测数据。

In [ ]:
!cd demo && python3 import.py

## 生成训练用的特征矩阵
模型训练需要训练数据，以下是生成使用的的代码

* 训练特征矩阵生成脚本代码 https://github.com/4paradigm/DemoApps/blob/main/predict-remaining-useful-life-nb/demo/gene_train_by_spark.py
* 训练数据使用`train_FD004.txt`，多engine不同cycle的检测数据。

整个任务最终会生成训练用的特征矩阵，保存为train_fm.csv。

### 特征详解
特征可分为两类，一类是一个engine的聚合特征，用sql可以表达为
```
select engine_no, MAX(operational_setting_1),... from d2 group by engine_no;
```
二类是相同time_in_cycles的所有engine的共同聚合特征，用sql可以表达为
```
select time_in_cycles, MAX(operational_setting_1) as rcm_operational_setting_1, ... from d2 group by time_in_cycles;""")

```


In [ ]:
!cd demo && sh train_fm.sh

## 使用训练的模型搭建链接fedb的实时推理http服务

基于上一步生成的训练用特征矩阵，使用`RandomForestRegressor`进行训练。
得到的训练模型，再结合fedb中的历史数据，搭建一个实时推理服务，整个推理服务代码参考https://github.com/4paradigm/DemoApps/blob/main/predict-remaining-useful-life-nb/demo/predict_server.py 。

In [ ]:
!cd demo && sh start_predict_server.sh
import os
output = os.popen('ps -ef | grep predict && kill -9 4791')
# output = os.popen('cat /tmp/p.log')
print(output.read())
# !cd demo && python3 predict_server.py

## 通过http请求发送一个推理请求

In [ ]:
!cd demo && python3 predict.py